In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
%matplotlib inline

# For visualization
import os
import seaborn as sns
from matplotlib import colors
from PIL import Image
from tqdm import tqdm

In [ ]:
!pip install pyngrok
!pip install ultralytics
!pip install grad-cam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=5b79e639e5aa1634a7a9bca12783f1d1e53849be01313fba08fb9df38b8d1f29
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
Successfully built grad-cam


In [ ]:
!pip install mmengine mmcv==2.1.0
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.4/471.4 kB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 27.4 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-2.1.0-cp312-cp312-linux_x86_64.whl size=39904683 sha256=224fb8504b54867ccb4d81158ecea61e32c3c9841af21063f7bcf55a985f20ac
  Stored in directory: /root/.cache/pip/wheels/e9/ed/f4/c494e3ff8fc7f14a8052d170fb3071cd96b7f825274b27df86
Successfully built mmcv
Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Total 38023 (delta 0), reused 0 (delta 0), pack-reused 38023 (from 1)
Receiving objects: 100% (38023/38023), 63.19 MiB | 15.32 MiB/s, done.
Resolving deltas: 100% (26217/26217), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Preparing metadata 

In [ ]:
!ngrok config add-authtoken 30n6o2UhSZEzRu8SnDRD6GetX2B_69sPW9jpcvFj86vjaZLDw


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Check Pytorch installation
import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())
print(torch.version.cuda)

# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)

# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)

# Check mmengine installation
import mmengine
print("mmengine:",mmengine.__version__)


torch version: 2.8.0+cu126 cuda: True
12.6
mmdetection: 3.3.0
mmcv: 2.1.0
mmengine: 0.10.7


In [ ]:
%cd ..

/content


In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import numpy as np
import torch
from ultralytics import YOLO
import cv2
import base64
import io
from io import BytesIO
import isCarRealModel as is_car_real

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

from torch.utils.data import Dataset
from pycocotools.coco import COCO
from mmengine import Config
from mmengine.runner import set_random_seed
import mmcv
from mmdet.registry import VISUALIZERS
from mmdet.apis import init_detector ,inference_detector
from mmengine.structures import InstanceData
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Implement updates necessary so webhooks use `public_url` from ngrok
app = Flask(__name__)

# Load car detection / count model (yolo11)
model_car_count = YOLO("yolo11x.pt")
model_car_count_classes = {"car", "bus", "truck"}
model_car_count_classes_id = [i for i, name in model_car_count.names.items() if name in model_car_count_classes]

# Load is car real model and def needed functions for that
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Instantiate model
model_is_car_real = is_car_real.IsCarRealModel_resnet152(pretrained=True)
model_is_car_real = model_is_car_real.to(device)

# Load best model (map to correct device)
model_is_car_real.load_state_dict(torch.load('./drive/MyDrive/API/best_car_real_model_resnet152.pth', map_location=device))
model_is_car_real.to(device)


def predict_image(pil_image, model, device):
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((480, 640)),
        transforms.ToTensor(),
    ])

    image = transform(pil_image.convert('RGB')).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        prob = torch.sigmoid(output).item()
        prediction = 1 if prob >= 0.5 else 0

    return prediction, prob


# Grad cam def for predict 2 incase = 0
def get_canny_edge(img, threshold1=30, threshold2=80):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = gray * 255
    gray = gray.astype(np.uint8)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edge = 255 - cv2.Canny(gray, threshold1, threshold2)
    edge = np.stack([edge] * 3, axis=-1) / 255
    return edge


class WrappedBinaryModel(torch.nn.Module):

    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, x):
        logit = self.base_model(x)  # shape [B, 1]
        logit = logit.view(-1, 1)
        two_logits = torch.cat([-logit, logit], dim=1)  # fake class 0 and 1
        return two_logits


def visualize_grad_cam(image_pil, model, target_layer, device):
    transform = transforms.Compose([
        transforms.Resize((480, 640)),
        transforms.ToTensor()
    ])

    image_tensor = transform(image_pil).unsqueeze(0).to(device)
    rgb_image = np.array(image_pil.resize((224, 224))) / 255.0

    edge_map = get_canny_edge((rgb_image * 255).astype(np.uint8))

    # Wrap the model for Grad-CAM compatibility
    wrapped_model = WrappedBinaryModel(model)
    cam = GradCAM(model=wrapped_model, target_layers=[target_layer])

    heatmaps = {}
    for class_id in [0]:
        targets = [ClassifierOutputTarget(class_id)]
        grayscale_cam = cam(input_tensor=image_tensor, targets=targets)[0]
        grayscale_cam_resized = cv2.resize(grayscale_cam, (edge_map.shape[1], edge_map.shape[0]),
                                           interpolation=cv2.INTER_LINEAR)
        heatmap = show_cam_on_image(edge_map, grayscale_cam_resized, use_rgb=True)
        heatmaps[f"class_{class_id}"] = heatmap

    # Combine visualizations
    composite = np.concatenate([
        (rgb_image * 255).astype(np.uint8),
        heatmaps["class_0"]
    ], axis=1)

    composite_bgr = cv2.cvtColor(composite, cv2.COLOR_RGB2BGR)

    # Convert to base64
    img_encode2 = cv2.imencode('.jpg', composite_bgr)[1]
    return base64.b64encode(img_encode2).decode('utf-8')


# model 3 detect car damages
config_file = './drive/MyDrive/API/v2_albumentations_cascade-mask-rcnn_x101_cfg.py'
checkpoint_file = './drive/MyDrive/API/epoch_24_weights_only.pth'
# Build the model from a config file and a checkpoint file
model_damage_detection = init_detector(config_file, checkpoint_file, device=0)


def inference_on_image(img_rgb_np, model, threshold=0.7):
    # Load and convert image
    img = mmcv.imconvert(img_rgb_np, 'bgr', 'rgb')

    visualizer = VISUALIZERS.build(model.cfg.visualizer)
    visualizer.dataset_meta = {
        'classes': ('dent', 'scratch', 'crack', 'glass shatter', 'lamp broken', 'tire flat'),
        'palette': [(220, 20, 60), (119, 11, 32), (0, 0, 142),
                    (0, 0, 230), (106, 0, 228), (0, 60, 100)]
    }

    result = inference_detector(model, img)
    predictions = result.pred_instances

    # Filter predictions by threshold
    indices = predictions.scores >= threshold
    filtered_masks = predictions.masks[indices]
    filtered_bboxes = predictions.bboxes[indices]
    filtered_labels = predictions.labels[indices]
    filtered_scores = predictions.scores[indices]

    filtered_predictions = InstanceData(
        masks=filtered_masks,
        bboxes=filtered_bboxes,
        labels=filtered_labels,
        scores=filtered_scores
    )
    result.pred_instances = filtered_predictions

    # visualizer
    visualizer.add_datasample(
        'result',
        img,
        data_sample=result,
        draw_gt=False,
        show=False
    )
    frame = visualizer.get_image()

    # Convert visualization to base64
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_frame)
    buffer = BytesIO()
    pil_img.save(buffer, format='JPEG')
    img_encode2 = base64.b64encode(buffer.getvalue()).decode('utf-8')

    # damage info
    classes = visualizer.dataset_meta['classes']
    detections = []
    for bbox, score, label in zip(filtered_predictions.bboxes, filtered_predictions.scores,
                                  filtered_predictions.labels):
        detections.append({
            'class': classes[label],
            'score': float(score),
            'bbox': [float(x) for x in bbox]
        })

    return {
        'total_detections': len(detections),
        'detections': detections,
        'visualization_base64': img_encode2
    }


# API def places
@app.route('/predict1', methods=['POST'])
def predict1():
    try:
        # Get JSON input0
        data = request.get_json()
        if 'base64' not in data:
            return jsonify({'error': 'No base64 field provided'}), 400

        # Decode base64 to image
        image_data = base64.b64decode(data['base64'])
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        image_np_rgb = np.array(image)

        # Predict with YOLO
        results = model_car_count.predict(image_np_rgb)

        # Count and draw boxes only for car, bus, truck
        vehicle_count = 0
        image_bgr = cv2.cvtColor(image_np_rgb, cv2.COLOR_RGB2BGR)
        for box in results[0].boxes:
            cls_id = int(box.cls)
            if cls_id in model_car_count_classes_id:
                vehicle_count += 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = model_car_count.names[cls_id]
                cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image_bgr, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Encode the result image with bounding boxes to base64
        img_encode = cv2.imencode('.jpg', image_bgr)[1]
        result_image_base64 = base64.b64encode(img_encode).decode('utf-8')

        return jsonify({
            'vehicle_count': vehicle_count,
            'result_image_base64': result_image_base64
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/predict2', methods=['POST'])
def predict2():
    try:
        # Get JSON input
        data = request.get_json()
        if 'base64' not in data:
            return jsonify({'error': 'No base64 field provided'}), 400

        # Decode base64 to image
        image_data = base64.b64decode(data['base64'])
        image = Image.open(io.BytesIO(image_data)).convert('RGB')

        # Predict with our model {'spoof': 0, 'real': 1}
        prediction, prob = predict_image(image, model_is_car_real, device)

        response = {
            'class': int(prediction),
            'probability': round(prob, 4)
        }

        # Grad cam if prediction class = not real
        target_layer = model_is_car_real.resnet.layer4[-1]
        if prediction == 0:
            grad_cam_base64 = visualize_grad_cam(image, model_is_car_real, target_layer, device)
            response['grad_cam_image_base64'] = grad_cam_base64

        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/predict3', methods=['POST'])
def predict3():
    try:
        # Get JSON input
        data = request.get_json()
        if 'base64' not in data:
            return jsonify({'error': 'No base64 field provided'}), 400

        # Decode base64 to image
        image_data = base64.b64decode(data['base64'])
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        image_np_rgb = np.array(image)

        result1 = inference_on_image(image_np_rgb, model_damage_detection, threshold=0.7)

        # Predict with YOLO
        image_bgr = cv2.cvtColor(image_np_rgb, cv2.COLOR_RGB2BGR)
        results = model_car_count.predict(image_np_rgb)
        boxes = results[0].boxes

        vehicle_box = None
        for box in boxes:
            cls_id = int(box.cls)
            if cls_id in model_car_count_classes_id:
                # Get box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # Take the first matching vehicle box
                vehicle_box = (x1, y1, x2, y2)
                # since "there will always be one"
                break

        x1, y1, x2, y2 = vehicle_box
        cropped_vehicle = image_np_rgb[y1:y2, x1:x2]

        # Run damage detection on the cropped car
        result2 = inference_on_image(cropped_vehicle, model_damage_detection, threshold=0.7)

        return jsonify({
            'full_image_damage': result1,
            'cropped_vehicle_damage': result2,
            'vehicle_box': [x1, y1, x2, y2]
        }), 200

    except Exception as e:
        print(e)
        return jsonify({'error': str(e)}), 500
# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://6409087e5fb5.ngrok-free.app" -> "http://127.0.0.1:5000"


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.12/dist-packages/mmengine/utils/dl_utils/misc.py:109: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  ext_loader = pkgutil.find_loader('mmcv._ext')


Loads checkpoint by local backend from path: ./drive/MyDrive/API/epoch_24_weights_only.pth
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer1.0.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer1.1.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer1.2.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer2.0.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer2.1.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer2.2.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer2.3.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer3.0.conv2 is upgraded to version 2.
08/20 17:20:14 - mmengine - INFO - DeformConv2dPack backbone.layer3.1.conv2 is upgraded to ve

/content/mmdetection/mmdet/apis/inference.py:90: UserWarning: dataset_meta or class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:

ngrok.kill()